# what is specific to TOPS SLC products?
Contrarily to GRD products SLC ones preserve the signal described in time per burst. 

It means that a portion of SAR image can be seen up to 4 times by the sensor.

The different bursts are overlapping, and the subswaths are also overlapping.

SLC products also contains the phase information, and digital number in .tiff products are complex values.

# How to read a TOPS SLC product with xsar ?

the TOPS (IW and EW) SLC products are distributed by ESA in .SAFE directories containing measurement and annotations per subswath and polarizations.

## to open a multi dataset

In [ ]:
import xsar
import geoviews as gv
import holoviews as hv
import geoviews.feature as gf
hv.extension('bokeh')
path = xsar.get_test_file('S1A_IW_SLC__1SDV_20170907T102951_20170907T103021_018268_01EB76_Z020.SAFE')
sub_swath_IDs = xsar.Sentinel1Meta(path).subdatasets
sub_swath_IDs

In [ ]:
multids = xsar.Sentinel1Meta(path)
#print(dir(multids.subdatasets))
for subswath in multids.subdatasets.index:
    print(subswath)
    onesubswath = xsar.Sentinel1Dataset(subswath)
    print(onesubswath.get_bursts_polygons()['geometry'])

In [ ]:
#gv.tile_sources.Wikipedia*gv.Polygons(multids.bursts(only_valid_location=True)['geometry']).opts(width=800,height=450,alpha=0.5)
import pandas as pd
tmp = pd.concat([xsar.Sentinel1Dataset(onesubswath).get_bursts_polygons(only_valid_location=True) for onesubswath in multids.subdatasets.index])
gv.tile_sources.EsriImagery*gv.Polygons(tmp['geometry']).opts(width=800,height=450,alpha=0.5)


In [ ]:
tmp2 = pd.concat([xsar.Sentinel1Dataset(onesubswath).get_bursts_polygons(only_valid_location=False) for onesubswath in multids.subdatasets.index])
gv.tile_sources.EsriImagery*gv.Polygons(tmp2['geometry']).opts(width=800,height=450,alpha=0.2)*gv.Polygons(tmp['geometry']).opts(width=800,height=450,alpha=0.2)


## to open a specific subswath
to open a subswath one can use [xsar.Sentinel1Meta](../basic_api.rst#xsar.Sentinel1Meta) class to get an overview of the meta-data.

In [ ]:
s1meta = xsar.sentinel1_meta.Sentinel1Meta("SENTINEL1_DS:%s:IW3" % path)
s1meta

to manipulate the data, the user have to open `Sentinel1Dataset` instance.

In [ ]:
s1ds = xsar.sentinel1_dataset.Sentinel1Dataset("SENTINEL1_DS:%s:IW3" % path)
s1ds

## content of the Sentinel1Dataset python object

 - a datatree (xarray)
 - a dataset (xarray)
 - a s1meta object (class instance)

## open the datatree

In [ ]:
s1ds.datatree

## open dataset

In [ ]:
s1ds.dataset

## get the sar_meta

In [ ]:
s1ds.sar_meta

# add high resolution interpolated variables

for instance longitudes, latitudes, incidence angle,...

Note that this step is automatically done by default for GRD products but not for SLC.

In [ ]:
s1ds.add_high_resolution_variables()
s1ds.dataset

# alternatively compute the value of a geolocation field (longitude, latitude, incidence,..) at given image coordinates
this method has been added to avoid adding hig resolution grids for some variable while we only need specific points

In [ ]:
s1ds.get_ll_from_SLC_geoloc(line=5,sample=3000,varname='incidence')

# perform calibration and denoizing on sigma0, gamma0 and beta0

In [ ]:
s1ds.apply_calibration_and_denoising() 
s1ds.dataset

In [ ]:
s1ds.dataset['ground_heading']

# get azimuth time variable 
the variable `azimuth_time` is given in annotations .xml files, it describes the date of acquisition of each pixel in the subswath. it is also called SAR "long time", in opposition to the "short time" given by the `slant_range_time` variable

In [ ]:
aziHr = s1ds.dataset['time']
hv.Curve(aziHr.values)

`azimuth time` variable estimated at the middle (in range) of the dataset selected is showing the bursts overlapping.

This variable is used to rasterize the 7 variables (longitude, latitude,...) described in the `geolocationGrid` at low resolution.

# get the ground range spacing
One specificity of the SLC products is that the ground range spacing is not equal to the slant range spacing (it is the case in GRD products).


In [ ]:
#print(s1ds.sar_meta.image['ground_pixel_spacing'])
#print(s1ds.sar_meta.image['slant_pixel_spacing'])
print(s1ds.dataset['sampleSpacing'])
print(s1ds.dataset['lineSpacing'])

The ground range spacing depends of the incidence angle.

$$ grdRangeSpacing = \frac{slantRangeSpacing}{sinus(\theta)} $$



It is possible for the users to get the ground range spacing vector along the range axis.



In [ ]:
rgs = s1ds.dataset['range_ground_spacing']
rgs

In [ ]:
hv.Curve(rgs).opts(width=400,show_grid=True)


# get complex digital number

In [ ]:
s1ds.datatree['measurement']['digital_number']

equivalent to

In [ ]:
s1ds.dataset['digital_number']

# additional informations

 - deramping TOPS SLC :[https://sentinels.copernicus.eu](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-1-sar/document-library/-/asset_publisher/1dO7RF5fJMbd/content/definition-of-the-tops-slc-deramping-function-for-products-generated-by-the-sentinel-1-ipf;jsessionid=DCEF041CCD5D10A93C637B6121D4D062.jvm1?redirect=https%3A%2F%2Fsentinels.copernicus.eu%2Fweb%2Fsentinel%2Fuser-guides%2Fsentinel-1-sar%2Fdocument-library%3Bjsessionid%3DDCEF041CCD5D10A93C637B6121D4D062.jvm1%3Fp_p_id%3D101_INSTANCE_1dO7RF5fJMbd%26p_p_lifecycle%3D0%26p_p_state%3Dnormal%26p_p_mode%3Dview%26p_p_col_id%3Dcolumn-1%26p_p_col_count%3D1)
 - TOPS technic: https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/topsar-processing
 - SLC products: https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/single-look-complex 